# Fine-tuning GPT-2 on a twitter dataset in PyTorch

In [2]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import Dataset, DataLoader
import os
import csv

#If using colab unhash
#from google.colab import drive
#import logging
#logging.getLogger().setLevel(logging.CRITICAL)

import warnings
warnings.filterwarnings('ignore')

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model = model.to(device)

In [4]:
def choose_from_top(probs, n=5):
    ind = np.argpartition(probs, -n)[-n:]
    top_prob = probs[ind]
    top_prob = top_prob / np.sum(top_prob) # Normalize
    choice = np.random.choice(n, 1, p = top_prob)
    token_id = ind[choice][0]
    return int(token_id)

### PyTorch Dataset module for tweets dataset

In [6]:
class TweetsDataset(Dataset):
    def __init__(self, tweets_dataset_path = ''):
        super().__init__()

        tweets_path = os.path.join(tweets_dataset_path, 'data_twq.csv')

        self.tweet_list = []
        self.end_of_text_token = "<|endoftext|>"

        with open(tweets_path, encoding="utf8") as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=';')
            x = 0
            for row in csv_reader:
                tweet_str = f"{row[1]}{self.end_of_text_token}"
                self.tweet_list.append(tweet_str)

    def __len__(self):
        return len(self.tweet_list)

    def __getitem__(self, item):
        return self.tweet_list[item]

In [7]:
dataset = TweetsDataset()
tweet_loader = DataLoader(dataset, batch_size=1, shuffle=True)

### Hyperparameters
If you run out of memory lower learning rate and/or lower batch_size

In [8]:
BATCH_SIZE = 16
EPOCHS = 5
LEARNING_RATE = 1e-3
WARMUP_STEPS = 5000
MAX_SEQ_LEN = 300
from transformers import AdamW, get_linear_schedule_with_warmup

device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'

### Model training

I will train the model and save the model weights after each epoch and then I will try to generate tweets with each version of the weight to see which performs the best.

In [13]:
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps  = -1)
proc_seq_count = 0
sum_loss = 0.0
batch_count = 0

tmp_tweets_tens = None
models_folder = "../trained_models"
if not os.path.exists(models_folder):
    os.mkdir(models_folder)

for epoch in range(EPOCHS):

    print(f"EPOCH {epoch} started" + '=' * 30)

    for idx,tweet in enumerate(tweet_loader):

        #################### "Fit as many tweet sequences into MAX_SEQ_LEN sequence as possible" logic start ####
        tweet_tens = torch.tensor(tokenizer.encode(tweet[0])).unsqueeze(0).to(device)
        #Skip sample from dataset if it is longer than MAX_SEQ_LEN
        if tweet_tens.size()[1] > MAX_SEQ_LEN:
            continue

        #The first tweet sequence in the sequence
        if not torch.is_tensor(tmp_tweets_tens):
            tmp_tweets_tens = tweet_tens
            continue
        else:
            #The next tweet does not fit in so we process the sequence and leave the last tweet
            #as the start for next sequence
            if tmp_tweets_tens.size()[1] + tweet_tens.size()[1] > MAX_SEQ_LEN:
                work_tweets_tens = tmp_tweets_tens
                tmp_tweets_tens = tweet_tens
            else:
                #Add the tweet to sequence, continue and try to add more
                tmp_tweets_tens = torch.cat([tmp_tweets_tens, tweet_tens[:,1:]], dim=1)
                continue
        ################## Sequence ready, process it trough the model ##################

        outputs = model(work_tweets_tens, labels=work_tweets_tens)
        loss, logits = outputs[:2]
        loss.backward()
        sum_loss = sum_loss + loss.detach().data

        proc_seq_count = proc_seq_count + 1
        if proc_seq_count == BATCH_SIZE:
            proc_seq_count = 0
            batch_count += 1
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            model.zero_grad()

        if batch_count == 10:
            print(f"sum loss {sum_loss}")
            batch_count = 0
            sum_loss = 0.0

    # Store the model after each epoch to compare the performance of them
    torch.save(model.state_dict(), os.path.join(models_folder, f"gpt2_xl_manbot_{epoch}.pt"))


EPOCH 0 started==============================


KeyboardInterrupt: 

### Generating the tweets

### Test the model with sample generated content that starts with word: "Tips"

In [10]:
MODEL_EPOCH = 0

models_folder = "../trained_models"

model_path = os.path.join(models_folder, f"gpt2_xl_manbot_{MODEL_EPOCH}.pt")
model.load_state_dict(torch.load(model_path))

tweets_output_file_path = f'generated_content_{MODEL_EPOCH}.txt'

model.eval()
if os.path.exists(tweets_output_file_path):
    os.remove(tweets_output_file_path)

tweet_num = 0
with torch.no_grad():

        for tweet_idx in range(20):

            tweet_finished = False

            cur_ids = torch.tensor(tokenizer.encode("Tips ")).unsqueeze(0).to(device)

            for i in range(100):
                outputs = model(cur_ids, labels=cur_ids)
                loss, logits = outputs[:2]
                softmax_logits = torch.softmax(logits[0,-1], dim=0) #Take the first(from only one in this case) batch and the last predicted embedding
                if i < 3:
                    n = 20
                else:
                    n = 3
                next_token_id = choose_from_top(softmax_logits.to('cpu').numpy(), n=n) #Randomly(from the topN probability distribution) select the next word
                cur_ids = torch.cat([cur_ids, torch.ones((1,1)).long().to(device) * next_token_id], dim = 1) # Add the last word to the running sequence

                if next_token_id in tokenizer.encode('<|endoftext|>'):
                    tweet_finished = True
                    break

            if tweet_finished:

                tweet_num = tweet_num + 1

                output_list = list(cur_ids.squeeze().to('cpu').numpy())
                output_text = tokenizer.decode(output_list)

                with open(tweets_output_file_path, 'a') as f:
                    f.write(f"{output_text} \n\n")